# 🏥 Analyse Avancée avec Dataset Enrichi - Clinique du Mont Vert
## IA Prédictive avec Prophet + Regressors Externes

---

**Version ENRICHI - Optimisée pour Prophet Avancé**

**Projet** : Master EISI - Gestion des stocks  
**Dataset** : Enrichi v3.0 - 5 ans de données (2020-2024)  
**Lignes** : 85,809 enregistrements  
**Colonnes** : 22 (15 base + 7 régresseurs)

---

## 📋 Nouveautés du Dataset Enrichi

### 🎯 7 Régresseurs Externes
- **temperature** : Température extérieure (°C)
- **taux_occupation** : Taux d'occupation hôpital (%)
- **nb_patients** : Nombre de patients
- **epidemie_grippe** : Périodes d'épidémie (0/1)
- **vacances_scolaires** : Vacances scolaires (0/1)
- **jour_ferie** : Jours fériés français (0/1)
- **covid_impact** : Périodes COVID (0/1)

### 🔄 Changepoints Majeurs
- **15/03/2020** : COVID-19 Vague 1 (+50%)
- **01/11/2020** : COVID-19 Vague 2 (+30%)
- **01/05/2021** : Déconfinement (-10%)
- **01/01/2022** : Nouvelle Direction (+10%)
- **01/09/2023** : Extension Hôpital (+15%)

### 📊 Résultats Attendus
- **MAE** : ~2.0 kg (vs 3.5 kg sans regressors)
- **MAPE** : ~20% (vs 35% sans regressors)
- **Précision** : Excellente

---

# 📦 Étape 1 : Imports et Configuration

In [ ]:
# Imports standards
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import json
from pathlib import Path
warnings.filterwarnings('ignore')

# Configuration pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Configuration matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (16, 8)
plt.rcParams['font.size'] = 10

print("✅ Imports de base terminés !")
print(f"📊 NumPy version : {np.__version__}")
print(f"📊 Pandas version : {pd.__version__}")

In [ ]:
# Import Prophet et utilitaires
print("🤖 Import de Prophet...")
try:
    from prophet import Prophet
    from prophet.plot import plot_yearly, plot_weekly, add_changepoints_to_plot
    from prophet.utilities import regressor_coefficients
    print("✅ Prophet importé avec succès !")
    PROPHET_AVAILABLE = True
except Exception as e:
    print(f"⚠️  Erreur Prophet : {e}")
    print("💡 Installation : pip install prophet")
    PROPHET_AVAILABLE = False

print(f"\n{'='*70}")
if PROPHET_AVAILABLE:
    print("🎉 Tout est prêt pour l'analyse avancée !")
else:
    print("⚠️  Prophet requis pour ce notebook")
print(f"{'='*70}")

In [ ]:
# Import Results Manager (optionnel)
try:
    from results_manager import ResultsManager
    results_mgr = ResultsManager()
    results_mgr.create_run_directory()
    print(f"✅ Results Manager activé")
    print(f"📁 Résultats seront sauvegardés dans : {results_mgr.get_run_path()}")
    USE_RESULTS_MANAGER = True
except:
    print("⚠️  Results Manager non disponible (optionnel)")
    USE_RESULTS_MANAGER = False

# 📂 Étape 2 : Chargement du Dataset Enrichi

In [ ]:
# Chemin du dataset enrichi
FICHIER_CSV = "../data/dataset_stock_hopital_ENRICHI.csv"

# Vérifier que le fichier existe
if not Path(FICHIER_CSV).exists():
    print(f"❌ Fichier introuvable : {FICHIER_CSV}")
    print("\n💡 Assurez-vous que le dataset enrichi est dans data/")
    raise FileNotFoundError(f"Le fichier {FICHIER_CSV} n'existe pas")

# Chargement
print(f"📊 Chargement de {FICHIER_CSV}...")
df = pd.read_csv(FICHIER_CSV)

print(f"✅ Dataset enrichi chargé avec succès !")
print(f"📏 Dimensions : {df.shape[0]:,} lignes × {df.shape[1]} colonnes")
print(f"💾 Taille mémoire : {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# Aperçu des données
print("👀 Aperçu des 5 premières lignes :")
df.head()

In [ ]:
# Vérifier les colonnes du dataset enrichi
print("📊 COLONNES DU DATASET ENRICHI")
print("="*70)

colonnes_base = ['date', 'id_produit', 'nom_produit', 'type_produit', 'type_operation',
                 'type_sortie', 'quantite', 'unite', 'id_lot', 'id_arrivage',
                 'id_fournisseur', 'nom_fournisseur', 'date_expiration',
                 'stock_theorique', 'temperature_stockage']

regresseurs = ['temperature', 'taux_occupation', 'nb_patients', 'epidemie_grippe',
               'vacances_scolaires', 'jour_ferie', 'covid_impact']

print("\n✅ Colonnes de base (15) :")
for col in colonnes_base:
    if col in df.columns:
        print(f"   ✓ {col}")
    else:
        print(f"   ✗ {col} (manquante)")

print("\n✅ Régresseurs externes (7) :")
for col in regresseurs:
    if col in df.columns:
        print(f"   ✓ {col}")
    else:
        print(f"   ✗ {col} (manquante)")

print(f"\n📊 Total colonnes trouvées : {len(df.columns)}")

# 🧹 Étape 3 : Préparation des Données

In [ ]:
# Conversion des dates
print("📅 Conversion des colonnes de dates...")
df['date'] = pd.to_datetime(df['date'])
df['date_expiration'] = pd.to_datetime(df['date_expiration'])

print("✅ Dates converties !")
print(f"📆 Période couverte : {df['date'].min().date()} → {df['date'].max().date()}")
print(f"⏱️  Durée totale : {(df['date'].max() - df['date'].min()).days} jours")
print(f"📊 Soit {(df['date'].max() - df['date'].min()).days / 365:.1f} ans de données")

In [ ]:
# Statistiques sur les régresseurs
print("📊 STATISTIQUES DES RÉGRESSEURS")
print("="*70)

print(f"\n🌡️  Température :")
print(f"   Moyenne : {df['temperature'].mean():.1f}°C")
print(f"   Min/Max : {df['temperature'].min():.1f}°C / {df['temperature'].max():.1f}°C")

print(f"\n🏥 Taux d'occupation :")
print(f"   Moyenne : {df['taux_occupation'].mean():.1f}%")
print(f"   Min/Max : {df['taux_occupation'].min():.1f}% / {df['taux_occupation'].max():.1f}%")

print(f"\n👥 Nombre de patients :")
print(f"   Moyenne : {df['nb_patients'].mean():.0f} patients/jour")
print(f"   Min/Max : {df['nb_patients'].min():.0f} / {df['nb_patients'].max():.0f}")

print(f"\n🦠 Épidémies de grippe :")
nb_jours_grippe = df['epidemie_grippe'].sum()
pct_grippe = (nb_jours_grippe / len(df) * 100)
print(f"   {nb_jours_grippe:,} jours d'épidémie ({pct_grippe:.1f}%)")

print(f"\n🏖️  Vacances scolaires :")
nb_jours_vacances = df['vacances_scolaires'].sum()
pct_vacances = (nb_jours_vacances / len(df) * 100)
print(f"   {nb_jours_vacances:,} jours de vacances ({pct_vacances:.1f}%)")

print(f"\n📅 Jours fériés :")
nb_jours_feries = df['jour_ferie'].sum()
nb_annees = (df['date'].max() - df['date'].min()).days / 365
print(f"   {nb_jours_feries:,} jours fériés (~{nb_jours_feries/nb_annees:.0f}/an)")

print(f"\n😷 Impact COVID :")
nb_jours_covid = df['covid_impact'].sum()
pct_covid = (nb_jours_covid / len(df) * 100)
print(f"   {nb_jours_covid:,} jours impactés ({pct_covid:.1f}%)")
print(f"   Soit {nb_jours_covid/30:.0f} mois cumulés")

# 🎯 Étape 4 : Sélection du Produit à Analyser

In [ ]:
# Choisir le produit à analyser
PRODUIT_ANALYSE = "Poulet frais"  # ← Changez ici pour analyser un autre produit

print(f"🔍 ANALYSE APPROFONDIE : {PRODUIT_ANALYSE}")
print("="*70)

# Filtrer uniquement les sorties de consommation (pas les destructions)
produit_df = df[
    (df['nom_produit'] == PRODUIT_ANALYSE) &
    (df['type_sortie'] == 'CONSOMMATION')
].copy()

if len(produit_df) == 0:
    print(f"❌ Aucune donnée trouvée pour '{PRODUIT_ANALYSE}'")
    print("\n💡 Produits disponibles :")
    print(df['nom_produit'].unique())
else:
    print(f"📊 {len(produit_df):,} sorties enregistrées")
    print(f"📅 Période : {produit_df['date'].min().date()} → {produit_df['date'].max().date()}")
    print(f"📦 Volume total : {produit_df['quantite'].sum():,.2f} kg")
    print(f"📈 Moyenne/sortie : {produit_df['quantite'].mean():.2f} kg")

# 📊 Étape 5 : Agrégation Quotidienne avec Régresseurs

In [ ]:
# Agréger par jour en gardant les régresseurs
print("🔧 Agrégation des données par jour...")

daily = produit_df.groupby('date').agg({
    'quantite': 'sum',                    # Somme des quantités consommées
    'temperature': 'mean',                # Moyenne température du jour
    'taux_occupation': 'mean',            # Moyenne taux occupation
    'nb_patients': 'mean',                # Moyenne nb patients
    'epidemie_grippe': 'max',             # 1 si épidémie ce jour
    'vacances_scolaires': 'max',          # 1 si vacances ce jour
    'jour_ferie': 'max',                  # 1 si férié ce jour
    'covid_impact': 'max'                 # 1 si COVID ce jour
}).reset_index()

print(f"✅ {len(daily)} jours avec données")

# Compléter les dates manquantes avec 0
print("🔧 Complétion des dates manquantes...")
date_range = pd.date_range(
    start=daily['date'].min(),
    end=daily['date'].max(),
    freq='D'
)
full_dates = pd.DataFrame({'date': date_range})
daily = full_dates.merge(daily, on='date', how='left')

# Remplir les valeurs manquantes
daily['quantite'].fillna(0, inplace=True)  # Pas de consommation = 0

# Pour les régresseurs continus : utiliser la moyenne
for col in ['temperature', 'taux_occupation', 'nb_patients']:
    daily[col].fillna(daily[col].mean(), inplace=True)

# Pour les régresseurs binaires : utiliser 0
for col in ['epidemie_grippe', 'vacances_scolaires', 'jour_ferie', 'covid_impact']:
    daily[col].fillna(0, inplace=True)

print(f"✅ {len(daily)} jours préparés (toutes les dates)")
print(f"   - Jours avec consommation : {(daily['quantite'] > 0).sum()}")
print(f"   - Jours sans consommation : {(daily['quantite'] == 0).sum()}")

In [ ]:
# Préparer le DataFrame pour Prophet
print("🔧 Préparation pour Prophet (renommage ds/y)...")

prophet_df = daily.copy()
prophet_df = prophet_df.rename(columns={'date': 'ds', 'quantite': 'y'})

print(f"✅ DataFrame Prophet prêt")
print(f"   - ds (date) : {prophet_df['ds'].min()} → {prophet_df['ds'].max()}")
print(f"   - y (quantité) : moyenne = {prophet_df['y'].mean():.2f} kg/jour")
print(f"   - Régresseurs : {', '.join(['temperature', 'taux_occupation', 'nb_patients', 'epidemie_grippe'])}")

# 📅 Étape 6 : Configuration des Holidays (Jours Fériés)

Prophet permet de modéliser des événements spéciaux via le système de "holidays".
Nous allons créer **3 types de holidays** :

1. **Jours fériés français** (Noël, 14 juillet, etc.)
2. **Vacances scolaires** (zones A, B, C)
3. **Périodes COVID** (événement majeur exceptionnel)

Ces événements ont un impact important sur la consommation hospitalière :
- Les jours fériés = moins de personnel = moins de consommation
- Les vacances = moins d'admissions = consommation réduite
- COVID = confinement + surcharge = consommation augmentée

In [ ]:
# Créer le DataFrame des jours fériés
print("📅 CONFIGURATION DES HOLIDAYS")
print("="*70)

# 1. Jours fériés français
holidays_jf = daily[daily['jour_ferie'] == 1][['date']].drop_duplicates()
holidays_jf.columns = ['ds']
holidays_jf['holiday'] = 'jour_ferie'
holidays_jf['lower_window'] = 0
holidays_jf['upper_window'] = 0

print(f"\n✅ Jours fériés : {len(holidays_jf)} jours")

# 2. Vacances scolaires
holidays_vac = daily[daily['vacances_scolaires'] == 1][['date']].drop_duplicates()
holidays_vac.columns = ['ds']
holidays_vac['holiday'] = 'vacances_scolaires'
holidays_vac['lower_window'] = 0
holidays_vac['upper_window'] = 0

print(f"✅ Vacances scolaires : {len(holidays_vac)} jours")

# 3. Périodes COVID (événement majeur)
holidays_covid = daily[daily['covid_impact'] == 1][['date']].drop_duplicates()
holidays_covid.columns = ['ds']
holidays_covid['holiday'] = 'covid_19'
holidays_covid['lower_window'] = 0
holidays_covid['upper_window'] = 0

print(f"✅ Périodes COVID : {len(holidays_covid)} jours")

# Combiner tous les holidays
holidays = pd.concat([holidays_jf, holidays_vac, holidays_covid])

print(f"\n📊 Total holidays configurés : {len(holidays)} jours")
print(f"\nTypes de holidays :")
print(holidays['holiday'].value_counts())

In [ ]:
# Aperçu des holidays
print("\n👀 Aperçu des 10 premiers holidays :")
holidays.head(10)

# 🔄 Étape 7 : Configuration des Changepoints

Les **changepoints** sont des moments où la tendance du modèle peut changer brusquement.

Prophet peut les détecter automatiquement, mais nous pouvons aussi les spécifier **manuellement** pour les événements importants que nous connaissons.

### 💡 Pourquoi des changepoints manuels ?

Dans notre dataset enrichi, nous connaissons 5 événements majeurs qui ont impacté la consommation :

| Date | Événement | Impact |
|------|-----------|--------|
| **15/03/2020** | COVID-19 Vague 1 | +50% consommation |
| **01/11/2020** | COVID-19 Vague 2 | +30% consommation |
| **01/05/2021** | Déconfinement | -10% consommation |
| **01/01/2022** | Nouvelle Direction | +10% consommation |
| **01/09/2023** | Extension Hôpital | +15% consommation |

En spécifiant ces dates, nous aidons Prophet à mieux capturer ces changements de tendance.

In [ ]:
# Définir les changepoints manuellement
print("🔄 CONFIGURATION DES CHANGEPOINTS")
print("="*70)

# Dates des événements majeurs (format YYYY-MM-DD)
changepoints_manuels = [
    '2020-03-15',  # COVID-19 Vague 1
    '2020-11-01',  # COVID-19 Vague 2
    '2021-05-01',  # Déconfinement
    '2022-01-01',  # Nouvelle Direction
    '2023-09-01'   # Extension Hôpital
]

print(f"\n✅ {len(changepoints_manuels)} changepoints manuels définis :")
for i, cp in enumerate(changepoints_manuels, 1):
    print(f"   {i}. {cp}")

print(f"\n💡 Note importante :")
print(f"   Prophet peut aussi détecter automatiquement d'autres changepoints")
print(f"   → Paramètre : changepoint_prior_scale")
print(f"   → Plus ce paramètre est élevé, plus le modèle est flexible")
print(f"   → Valeur par défaut : 0.05 (peu flexible)")
print(f"   → Valeur recommandée pour notre dataset : 0.5 (plus flexible)")

# 📊 Étape 8 : Split Train/Test

Pour évaluer la performance de notre modèle, nous devons diviser nos données en deux parties :

### 🎯 Principe du Split

- **Train (Entraînement)** : Données que le modèle va utiliser pour apprendre les patterns
- **Test (Validation)** : Données réservées pour évaluer la performance

### 📐 Notre Stratégie

Nous avons **5 ans de données** (2020-2024). Nous allons utiliser :

- **Train** : Les 4 premières années (~80% des données)
- **Test** : La dernière année (~20% des données)

### 💡 Pourquoi 1 an de test ?

- C'est suffisant pour capturer toutes les variations saisonnières (4 saisons complètes)
- Cela nous permet de tester la capacité du modèle à prédire sur une période longue
- C'est une pratique standard en time series forecasting

In [ ]:
# Split train/test
print("📊 SPLIT TRAIN/TEST")
print("="*70)

# Garder la dernière année pour le test (365 jours)
split_date = prophet_df['ds'].max() - pd.Timedelta(days=365)
train = prophet_df[prophet_df['ds'] <= split_date].copy()
test = prophet_df[prophet_df['ds'] > split_date].copy()

print(f"\n✅ Données divisées :")
print(f"   📅 Date de split : {split_date.date()}")
print(f"\n   🎓 Train : {len(train)} jours ({train['ds'].min().date()} → {train['ds'].max().date()})")
print(f"   🧪 Test  : {len(test)} jours ({test['ds'].min().date()} → {test['ds'].max().date()})")
print(f"\n   📊 Ratio : {len(train)/len(prophet_df)*100:.1f}% train / {len(test)/len(prophet_df)*100:.1f}% test")

# Vérifier que le test contient bien des régresseurs
print(f"\n✅ Vérification des régresseurs dans le test :")
print(f"   - Temperature : {test['temperature'].notna().sum()} valeurs")
print(f"   - Taux occupation : {test['taux_occupation'].notna().sum()} valeurs")
print(f"   - Nb patients : {test['nb_patients'].notna().sum()} valeurs")
print(f"   - Epidemie grippe : {test['epidemie_grippe'].notna().sum()} valeurs")

# 🤖 Étape 9 : Modèle Prophet avec Regressors

C'est l'étape la plus importante ! Nous allons configurer un modèle Prophet **avancé** avec toutes les fonctionnalités.

### 🎯 Configuration Complète

Notre modèle va intégrer :

1. **Holidays** → Jours fériés, vacances, COVID
2. **Changepoints** → 5 événements majeurs manuels
3. **Seasonality** → Saisonnalité annuelle (20) + hebdomadaire (5)
4. **Regressors** → 4 variables externes (température, occupation, patients, grippe)

### 📊 Paramètres Détaillés

| Paramètre | Valeur | Explication |
|-----------|--------|-------------|
| `yearly_seasonality` | 20 | Fourier order pour capturer les cycles annuels (default: 10) |
| `weekly_seasonality` | 5 | Fourier order pour les cycles hebdomadaires (default: 3) |
| `seasonality_mode` | multiplicative | Effet multiplicatif (mieux pour nos données) |
| `changepoint_prior_scale` | 0.5 | Flexibilité des changepoints (default: 0.05) |
| `holidays_prior_scale` | 10.0 | Importance des holidays (default: 10) |
| `interval_width` | 0.85 | Intervalle de confiance à 85% |

In [ ]:
# Configuration du modèle Prophet
print("🤖 CONFIGURATION DU MODÈLE PROPHET")
print("="*70)

# Créer le modèle avec configuration optimale
model = Prophet(
    # ===== HOLIDAYS =====
    holidays=holidays,                # Les 3 types configurés précédemment
    holidays_prior_scale=10.0,        # Importance des holidays (default: 10)
    
    # ===== SEASONALITY =====
    yearly_seasonality=20,            # Ordre Fourier annuel (default: 10, max: 20)
    weekly_seasonality=5,             # Ordre Fourier hebdo (default: 3)
    daily_seasonality=False,          # Pas nécessaire pour agrégation quotidienne
    seasonality_mode='multiplicative', # Effet multiplicatif (vs 'additive')
    seasonality_prior_scale=10.0,     # Flexibilité saisonnalité (default: 10)
    
    # ===== CHANGEPOINTS =====
    changepoints=changepoints_manuels, # 5 événements majeurs
    changepoint_prior_scale=0.5,      # Flexibilité (default: 0.05, plus = flexible)
    changepoint_range=0.9,            # 90% des données (default: 0.8)
    
    # ===== AUTRES =====
    interval_width=0.85,              # Intervalle de confiance 85%
    growth='linear',                  # Croissance linéaire (vs 'logistic')
    mcmc_samples=0                    # 0 = pas de MCMC (plus rapide)
)

print("\n✅ Modèle créé avec :")
print(f"   🎉 Holidays : {len(holidays)} événements")
print(f"   🔄 Changepoints : {len(changepoints_manuels)} manuels")
print(f"   📊 Seasonality : yearly (20) + weekly (5)")
print(f"   🎯 Mode : multiplicative")
print(f"   📈 Intervalle confiance : 85%")

In [ ]:
# Ajouter les régresseurs externes
print("\n🔧 AJOUT DES RÉGRESSEURS EXTERNES")
print("="*70)

# 1. Température (effet sur produits frais/périssables)
model.add_regressor(
    'temperature',
    prior_scale=0.5,      # Importance modérée
    standardize=True,     # Normaliser automatiquement (recommandé)
    mode='additive'       # Effet additif
)
print("✅ Régresseur 'temperature' ajouté")
print("   → Impact sur produits sensibles à la température")

# 2. Taux d'occupation (corrélation forte avec consommation)
model.add_regressor(
    'taux_occupation',
    prior_scale=1.0,      # Importance élevée (forte corrélation)
    standardize=True,
    mode='additive'
)
print("✅ Régresseur 'taux_occupation' ajouté")
print("   → Plus de patients = plus de consommation")

# 3. Nombre de patients (alternative au taux occupation)
model.add_regressor(
    'nb_patients',
    prior_scale=0.5,      # Importance modérée (redondant avec occupation)
    standardize=True,
    mode='additive'
)
print("✅ Régresseur 'nb_patients' ajouté")
print("   → Complément du taux d'occupation")

# 4. Épidémie de grippe (événement ponctuel à fort impact)
model.add_regressor(
    'epidemie_grippe',
    prior_scale=0.5,      # Importance modérée
    standardize=False,    # Déjà binaire (0 ou 1), pas besoin de normaliser
    mode='additive'
)
print("✅ Régresseur 'epidemie_grippe' ajouté")
print("   → Impact des épidémies hivernales")

print(f"\n📊 Total : 4 régresseurs externes configurés !")
print(f"💡 Ces variables vont améliorer significativement les prédictions")

In [ ]:
# Entraîner le modèle
print("\n⏳ ENTRAÎNEMENT DU MODÈLE...")
print("="*70)
print("⏳ Cela peut prendre 1-2 minutes selon votre machine...")
print("   Prophet utilise Stan (optimisation bayésienne)")

# Fit sur les données d'entraînement
model.fit(train)

print("\n✅ Modèle entraîné avec succès !")
print("   Le modèle a appris les patterns sur 4 ans de données")
print("   Nous allons maintenant l'évaluer sur l'année de test")

# 📊 Étape 10 : Évaluation sur le Test

Maintenant que le modèle est entraîné, nous allons évaluer sa **performance** sur les données de test (1 an).

### 📐 Métriques Utilisées

Nous utilisons 3 métriques standard pour évaluer la précision :

| Métrique | Nom Complet | Signification | Bon Score |
|----------|-------------|---------------|-----------|
| **MAE** | Mean Absolute Error | Erreur moyenne en kg | < 3 kg |
| **MAPE** | Mean Absolute Percentage Error | Erreur relative en % | < 20% |
| **RMSE** | Root Mean Squared Error | Pénalise les grandes erreurs | < 4 kg |

### 💡 Interprétation

- **MAE** : "En moyenne, je me trompe de X kg"
- **MAPE** : "En moyenne, je me trompe de X%"
- **RMSE** : "Mes erreurs les plus importantes sont de l'ordre de X kg"

In [ ]:
# Prédire sur les données de test
print("📊 ÉVALUATION SUR LE TEST")
print("="*70)

# Générer les prédictions pour la période de test
predictions_test = model.predict(test)

# Extraire les valeurs réelles et prédites
y_true = test['y'].values      # Vraies valeurs
y_pred = predictions_test['yhat'].values  # Prédictions

# Calculer les 3 métriques principales
mae = np.mean(np.abs(y_true - y_pred))
mape = np.mean(np.abs((y_true - y_pred) / (y_true + 0.01))) * 100  # +0.01 évite division par 0
rmse = np.sqrt(np.mean((y_true - y_pred)**2))

# Afficher les résultats
print(f"\n📊 MÉTRIQUES DE PERFORMANCE")
print("="*70)
print(f"MAE  (Erreur Absolue Moyenne)    : {mae:.2f} kg")
print(f"MAPE (Erreur Relative Moyenne)   : {mape:.2f}%")
print(f"RMSE (Erreur Quadratique Moyenne): {rmse:.2f} kg")
print("="*70)

# Interpréter les résultats
if mape < 15:
    verdict = "✅ Excellente précision ! (MAPE < 15%)"
    commentaire = "Le modèle est très performant. Résultats fiables."
elif mape < 25:
    verdict = "✅ Bonne précision (MAPE entre 15% et 25%)"
    commentaire = "Le modèle est satisfaisant. Peut être utilisé en production."
else:
    verdict = "⚠️  Précision moyenne (MAPE > 25%)"
    commentaire = "Le modèle pourrait être amélioré."

print(f"\n{verdict}")
print(f"💡 {commentaire}")
print(f"\n📈 En pratique : Le modèle se trompe en moyenne de ±{mae:.2f} kg (soit {mape:.1f}%)")

# 🔬 Étape 11 : Analyse des Coefficients des Régresseurs

Une des forces de Prophet est qu'il permet d'**interpréter** l'impact de chaque régresseur.

### 💡 Qu'est-ce qu'un coefficient ?

Le coefficient représente **l'impact** d'un régresseur sur la prédiction :

- **Coefficient positif** → Quand le régresseur augmente, la consommation augmente
- **Coefficient négatif** → Quand le régresseur augmente, la consommation diminue

### 📊 Exemple d'Interprétation

Si le coefficient de `taux_occupation` est **+0.25** :
- Quand le taux d'occupation augmente de 1%, la consommation augmente de ~0.25 kg
- C'est un **effet positif** : plus de patients = plus de consommation

### 🎯 Ce que nous allons analyser

- Impact de la **température** sur les produits frais
- Impact du **taux d'occupation** sur la consommation
- Impact du **nombre de patients**
- Impact des **épidémies de grippe**

In [ ]:
# Extraire les coefficients des régresseurs
print("🔬 ANALYSE DES COEFFICIENTS DES RÉGRESSEURS")
print("="*70)

try:
    # Utiliser la fonction native de Prophet
    coeffs = regressor_coefficients(model)
    
    print("\n📊 Tableau des coefficients :")
    print(coeffs.to_string(index=False))
    
    print("\n" + "="*70)
    print("💡 INTERPRÉTATION DÉTAILLÉE")
    print("="*70)
    
    # Interpréter chaque coefficient
    for idx, row in coeffs.iterrows():
        regressor = row['regressor']
        coeff = row['coef']
        coeff_lower = row['coef_lower']
        coeff_upper = row['coef_upper']
        
        # Déterminer le sens de l'effet
        if coeff > 0:
            effet = "📈 EFFET POSITIF"
            interpretation = "augmente la consommation"
        else:
            effet = "📉 EFFET NÉGATIF"
            interpretation = "diminue la consommation"
        
        print(f"\n🔹 {regressor.upper()}")
        print(f"   Coefficient : {coeff:+.4f}")
        print(f"   Intervalle  : [{coeff_lower:.4f}, {coeff_upper:.4f}]")
        print(f"   {effet} → {interpretation}")
        
        # Interprétation spécifique par régresseur
        if regressor == 'temperature':
            if coeff > 0:
                print(f"   💡 Plus il fait chaud, plus la consommation augmente (+{abs(coeff):.4f} kg/°C)")
            else:
                print(f"   💡 Plus il fait chaud, moins la consommation augmente ({coeff:.4f} kg/°C)")
        
        elif regressor == 'taux_occupation':
            if coeff > 0:
                print(f"   💡 1% d'occupation en plus = +{coeff:.4f} kg de consommation")
                print(f"   💡 Forte corrélation attendue : plus de patients = plus de repas")
        
        elif regressor == 'nb_patients':
            if coeff > 0:
                print(f"   💡 1 patient supplémentaire = +{coeff:.4f} kg")
        
        elif regressor == 'epidemie_grippe':
            if coeff > 0:
                print(f"   💡 En période d'épidémie : +{coeff:.4f} kg/jour")
            else:
                print(f"   💡 En période d'épidémie : {coeff:.4f} kg/jour (moins de consommation)")
    
    print("\n" + "="*70)
    print("✅ Analyse des coefficients terminée !")
    
except Exception as e:
    print(f"⚠️  Impossible d'extraire les coefficients : {e}")
    print("💡 Cela peut arriver si le modèle n'a pas convergé correctement")

# 🔮 Étape 12 : Prédictions Futures et Visualisations

Maintenant que notre modèle est validé, nous allons l'utiliser pour **prédire l'avenir** !

### 🎯 Objectif

Prédire la consommation pour les **28 prochains jours** (4 semaines) avec intervalles de confiance.

### 📊 Processus

1. **Réentraîner** le modèle sur **toutes les données** (train + test)
2. **Créer** les 28 jours futurs avec valeurs des régresseurs
3. **Prédire** la consommation future
4. **Visualiser** les résultats avec 3 graphiques professionnels

### 🔧 Gestion des Régresseurs Futurs

Pour les 28 jours futurs, nous devons fournir des valeurs pour nos 4 régresseurs :

| Régresseur | Stratégie |
|------------|-----------|
| `temperature` | Utiliser la moyenne historique |
| `taux_occupation` | Utiliser la moyenne historique |
| `nb_patients` | Utiliser la moyenne historique |
| `epidemie_grippe` | 1 si hiver (janvier-mars), 0 sinon |

In [ ]:
# Réentraîner sur TOUTES les données (train + test)
print("🔮 PRÉDICTIONS FUTURES (28 JOURS)")
print("="*70)

print("\n⏳ Réentraînement du modèle sur toutes les données...")

# Créer un nouveau modèle avec la même configuration
model_final = Prophet(
    holidays=holidays,
    holidays_prior_scale=10.0,
    yearly_seasonality=20,
    weekly_seasonality=5,
    daily_seasonality=False,
    seasonality_mode='multiplicative',
    seasonality_prior_scale=10.0,
    changepoints=changepoints_manuels,
    changepoint_prior_scale=0.5,
    changepoint_range=0.9,
    interval_width=0.85,
    growth='linear'
)

# Ajouter les mêmes régresseurs
model_final.add_regressor('temperature', prior_scale=0.5, standardize=True)
model_final.add_regressor('taux_occupation', prior_scale=1.0, standardize=True)
model_final.add_regressor('nb_patients', prior_scale=0.5, standardize=True)
model_final.add_regressor('epidemie_grippe', prior_scale=0.5, standardize=False)

# Fit sur TOUTES les données
model_final.fit(prophet_df)

print("✅ Modèle réentraîné sur toutes les données !")
print(f"   Données utilisées : {len(prophet_df)} jours")

In [ ]:
# Créer le DataFrame futur avec régresseurs
print("\n🔧 Création des 28 jours futurs avec régresseurs...")

# Créer les dates futures (28 jours)
future = model_final.make_future_dataframe(periods=28)

# Fusionner avec les régresseurs existants
future = future.merge(
    prophet_df[['ds', 'temperature', 'taux_occupation', 'nb_patients', 'epidemie_grippe']],
    on='ds',
    how='left'
)

# Pour les jours futurs (NaN), utiliser des valeurs par défaut
print("   Remplissage des régresseurs futurs...")

# Régresseurs continus : moyenne historique
for col in ['temperature', 'taux_occupation', 'nb_patients']:
    mean_value = prophet_df[col].mean()
    future[col].fillna(mean_value, inplace=True)
    print(f"   - {col} : {mean_value:.2f} (moyenne)")

# Épidémie de grippe : activer si hiver (janvier-mars)
future['epidemie_grippe'].fillna(
    future['ds'].dt.month.isin([1, 2, 3]).astype(int),
    inplace=True
)
nb_jours_epidemie_futur = future[future['ds'] > prophet_df['ds'].max()]['epidemie_grippe'].sum()
print(f"   - epidemie_grippe : {nb_jours_epidemie_futur} jours d'épidémie prévus")

print(f"\n✅ {len(future)} jours préparés (historique + 28 futurs)")

In [ ]:
# Générer les prédictions
print("\n⏳ Génération des prédictions...")

forecast = model_final.predict(future)

# Extraire uniquement les prédictions futures (28 jours)
predictions_futures = forecast[forecast['ds'] > prophet_df['ds'].max()].copy()

print(f"✅ {len(predictions_futures)} jours de prédictions générées !")
print(f"\n📊 STATISTIQUES DES PRÉDICTIONS")
print("="*70)
print(f"Total prévu sur 28 jours  : {predictions_futures['yhat'].sum():.2f} kg")
print(f"Moyenne par jour          : {predictions_futures['yhat'].mean():.2f} kg")
print(f"Minimum prévu             : {predictions_futures['yhat'].min():.2f} kg")
print(f"Maximum prévu             : {predictions_futures['yhat'].max():.2f} kg")
print(f"Écart-type                : {predictions_futures['yhat'].std():.2f} kg")
print("="*70)

In [ ]:
# Aperçu des prédictions
print("\n👀 Aperçu des 10 premiers jours de prédictions :")
predictions_futures[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head(10)

### 📈 Visualisations Professionnelles

Nous allons créer **3 graphiques** pour analyser les résultats :

1. **Graphique Principal** : Prédictions avec intervalle de confiance
2. **Composants** : Décomposition (tendance + saisonnalité + holidays)
3. **Changepoints** : Points de rupture de tendance

In [ ]:
# Graphique 1 : Prédictions principales
print("\n📈 GÉNÉRATION DES VISUALISATIONS")
print("="*70)

# Graphique principal avec prédictions
fig1 = model_final.plot(forecast)
plt.title(f'Prédictions Prophet - {PRODUIT_ANALYSE}', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Consommation (kg)', fontsize=12)
plt.tight_layout()

filename1 = f'predictions_{PRODUIT_ANALYSE.replace(" ", "_")}_enrichi.png'
plt.savefig(filename1, dpi=300, bbox_inches='tight')
print(f"✅ Graphique 1 sauvegardé : {filename1}")
plt.show()
plt.close()

In [ ]:
# Graphique 2 : Composants (décomposition)
fig2 = model_final.plot_components(forecast)
plt.tight_layout()

filename2 = f'components_{PRODUIT_ANALYSE.replace(" ", "_")}_enrichi.png'
plt.savefig(filename2, dpi=300, bbox_inches='tight')
print(f"✅ Graphique 2 sauvegardé : {filename2}")
print("   → Montre la tendance, saisonnalité annuelle, hebdomadaire et holidays")
plt.show()
plt.close()

In [ ]:
# Graphique 3 : Changepoints (points de rupture)
fig3 = model_final.plot(forecast)
add_changepoints_to_plot(fig3.gca(), model_final, forecast)
plt.title(f'Changepoints - {PRODUIT_ANALYSE}', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Consommation (kg)', fontsize=12)
plt.tight_layout()

filename3 = f'changepoints_{PRODUIT_ANALYSE.replace(" ", "_")}_enrichi.png'
plt.savefig(filename3, dpi=300, bbox_inches='tight')
print(f"✅ Graphique 3 sauvegardé : {filename3}")
print("   → Montre les 5 changepoints manuels + ceux détectés automatiquement")
plt.show()
plt.close()

print("\n" + "="*70)
print("✅ 3 graphiques générés avec succès !")
print("="*70)

# 💾 Étape 13 : Export des Résultats

Nous allons maintenant **exporter** tous les résultats dans des formats réutilisables :

### 📁 Fichiers à Exporter

1. **CSV** : Prédictions des 28 jours avec intervalles de confiance
2. **JSON** : Résumé complet (métriques + paramètres + recommandations)
3. **PNG** : Les 3 graphiques générés

### 🗂️ Organisation avec Results Manager

Si le Results Manager est disponible, tous les fichiers seront automatiquement organisés dans un dossier horodaté : `results/YYYYMMDD_HHMMSS/`

Cela permet de :
- Comparer plusieurs exécutions
- Conserver l'historique des prédictions
- Retrouver facilement les résultats

In [ ]:
# Export CSV des prédictions
print("💾 EXPORT DES RÉSULTATS")
print("="*70)

# Préparer le DataFrame d'export
export_df = predictions_futures[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
export_df.columns = ['date', 'quantite_prevue', 'quantite_min', 'quantite_max']
export_df['date'] = export_df['date'].dt.date
export_df['produit'] = PRODUIT_ANALYSE
export_df['confiance'] = '85%'

# Réorganiser les colonnes
export_df = export_df[['date', 'produit', 'quantite_prevue', 'quantite_min', 'quantite_max', 'confiance']]

# Sauvegarder
filename_csv = f'predictions_{PRODUIT_ANALYSE.replace(" ", "_")}_enrichi_28j.csv'
export_df.to_csv(filename_csv, index=False, encoding='utf-8')
print(f"✅ CSV sauvegardé : {filename_csv}")
print(f"   → {len(export_df)} jours de prédictions")

In [ ]:
# Export JSON du résumé
print("\n📄 Création du résumé JSON...")

summary = {
    "produit": PRODUIT_ANALYSE,
    "date_analyse": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    "dataset": {
        "fichier": "dataset_stock_hopital_ENRICHI.csv",
        "version": "3.0",
        "nb_lignes_total": len(df),
        "periode": f"{df['date'].min().date()} → {df['date'].max().date()}"
    },
    "performance_modele": {
        "MAE": round(mae, 2),
        "MAPE": round(mape, 2),
        "RMSE": round(rmse, 2),
        "methode": "Prophet avec regressors enrichis",
        "verdict": verdict
    },
    "configuration_modele": {
        "holidays": ["jour_ferie", "vacances_scolaires", "covid_19"],
        "nb_holidays": len(holidays),
        "changepoints_manuels": changepoints_manuels,
        "nb_changepoints": len(changepoints_manuels),
        "regresseurs": ["temperature", "taux_occupation", "nb_patients", "epidemie_grippe"],
        "seasonality": {
            "yearly": 20,
            "weekly": 5,
            "mode": "multiplicative"
        }
    },
    "predictions": {
        "horizon": "28 jours",
        "periode": f"{predictions_futures['ds'].min().date()} → {predictions_futures['ds'].max().date()}",
        "total_prevu": round(predictions_futures['yhat'].sum(), 2),
        "moyenne_jour": round(predictions_futures['yhat'].mean(), 2),
        "min_jour": round(predictions_futures['yhat'].min(), 2),
        "max_jour": round(predictions_futures['yhat'].max(), 2),
        "intervalle_confiance": "85%"
    },
    "recommandations": {
        "commande_hebdomadaire": round(predictions_futures['yhat'].mean() * 7, 2),
        "stock_securite": round(predictions_futures['yhat_upper'].max() * 1.2, 2),
        "attention": "Tenir compte des variations saisonnières"
    }
}

filename_json = f'summary_{PRODUIT_ANALYSE.replace(" ", "_")}_enrichi.json'
with open(filename_json, 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)

print(f"✅ JSON sauvegardé : {filename_json}")

In [ ]:
# Sauvegarder avec Results Manager si disponible
if USE_RESULTS_MANAGER:
    print("\n📁 Sauvegarde avec Results Manager...")
    
    # Sauvegarder les graphiques
    for filename in [filename1, filename2, filename3]:
        results_mgr.save_graph(filename)
        print(f"   ✅ {filename}")
    
    # Sauvegarder les données
    results_mgr.save_data(filename_csv)
    print(f"   ✅ {filename_csv}")
    
    results_mgr.save_data(filename_json)
    print(f"   ✅ {filename_json}")
    
    # Créer le fichier README automatique
    results_mgr.create_summary_file(PRODUIT_ANALYSE, summary)
    
    print(f"\n✅ Tous les résultats sauvegardés dans :")
    print(f"   📂 {results_mgr.get_run_path()}")
else:
    print("\n📁 Fichiers sauvegardés dans le répertoire courant")
    print(f"   📄 {filename_csv}")
    print(f"   📄 {filename_json}")
    print(f"   📊 {filename1}")
    print(f"   📊 {filename2}")
    print(f"   📊 {filename3}")

In [ ]:
# Afficher le résumé final
print("\n" + "="*70)
print("🎉 ANALYSE TERMINÉE AVEC SUCCÈS !")
print("="*70)

print(f"\n📊 RÉSUMÉ FINAL")
print("="*70)
print(f"Produit analysé       : {PRODUIT_ANALYSE}")
print(f"Dataset utilisé       : Enrichi v3.0 ({len(df):,} lignes)")
print(f"Modèle                : Prophet avec 4 régresseurs")
print(f"Performance (MAPE)    : {mape:.2f}%")
print(f"Prédictions générées  : 28 jours ({predictions_futures['yhat'].sum():.2f} kg total)")
print(f"Graphiques créés      : 3 visualisations PNG")
print(f"Fichiers exportés     : CSV + JSON")
print("="*70)

print(f"\n💡 RECOMMANDATIONS")
print("="*70)
print(f"✅ Commande hebdomadaire suggérée : {summary['recommandations']['commande_hebdomadaire']:.2f} kg")
print(f"✅ Stock de sécurité suggéré      : {summary['recommandations']['stock_securite']:.2f} kg")
print(f"⚠️  Attention : {summary['recommandations']['attention']}")
print("="*70)

print(f"\n🚀 PROCHAINES ÉTAPES")
print("   1. Analyser d'autres produits en changeant PRODUIT_ANALYSE")
print("   2. Comparer les résultats avec le dataset de base")
print("   3. Ajuster les paramètres si nécessaire")
print("   4. Mettre en production les prédictions")
print("\n✨ Notebook terminé ! Vous pouvez maintenant exploiter les résultats.")